In [1]:
import os
import logging
import boto3
import zipfile
import json
import io
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
from io import BytesIO

In [2]:
s3_host = 'https://obj.umiacs.umd.edu'
access_key_id = "xxxxx"
secret_access_key = "xxxxx"

s3 = boto3.client('s3',
                  endpoint_url=s3_host,
                  aws_access_key_id=access_key_id,
                  aws_secret_access_key=secret_access_key)

bucket = 'twitter.tmrc'


In [3]:
import ast

df = pd.read_csv('~/Coding/buntain/retweet_data/twitterei_user_dict.csv')

user_name_dict = df.set_index('userid')['twitterid'].to_dict()
user_name_dict = {key: ast.literal_eval(value) for key, value in user_name_dict.items()}

print(user_name_dict)
print(len(user_name_dict))
print(type(user_name_dict[206027550]))
for k in user_name_dict.keys():
    print(type(k))
    break

{206027550: ['nessniven'], 898517260033290240: ['2VB3DcLYobExE3v'], 620912741: ['qujianming1'], 791015242448850945: ['klausv02'], 769790067183190016: ['SamanthxBerg'], 902777994829336576: ['gHk39king'], 898581276063617025: ['y6Hcuj97hRQd9gl'], 1084444722: ['nurlanyc5sr'], 761109495187513344: ['SheldonxLill'], 910296636294877184: ['xiaolaoshubai'], 902806427995172865: ['cUiIRrOVbn2Ripa'], 34649696: ['kaiking89'], 718766229700227072: ['RanvijaySowle'], 920467059175124992: ['Gregory22A'], 902779178180890626: ['HRiB831Jam'], 760761981854883840: ['honyBurchr'], 1085775014270459906: ['bronte_megan'], 3388184553: ['lacy6ysjodin'], 912171517076807680: ['oksananikolenk5'], 747309343624704000: ['kotovaninel89'], 4057984825: ['baillopud5'], 910162184772059136: ['ailaiyi1'], 918004985513705473: ['Davio91Guerl'], 788221364: ['zhangbide9600'], 926731550313480192: ['yszdian'], 2705083230: ['12muchum'], 1017843752147222530: ['masminge'], 747356042246291456: ['tattazueva2'], 2329240622: ['kristinaeug5k

In [4]:
def process(folder_prefix, user_name_dict, mentions_dict, retweet_dict, suspended_users):
    response = s3.list_objects_v2(Bucket=bucket, Prefix=folder_prefix)
    i = 0
    bad_userid = 0
    files = []
    for obj in response.get('Contents', []):
        i += 1
        object_key = obj['Key']
        if object_key.endswith('.zip'):
            files.append(object_key)
            # print(f"Processing ZIP file: {object_key}")
    print(f"number of files: {i}")
    for file in files:
        zip_object = s3.get_object(Bucket=bucket, Key=file)
        zip_contents = zip_object['Body'].read()
        zip_file = zipfile.ZipFile(io.BytesIO(zip_contents), 'r')
        for file_info in zip_file.infolist():
            with zip_file.open(file_info) as json_file:
                file_name = file_info.filename
                if not file_name.endswith("-tweet.json"):
                    continue
                try:
                    json_data = json_file.read().decode('utf-8')
                except:
                    print("this is a text file")
                parsed_data = json.loads(json_data)
                for ind_data in parsed_data:
                    # if 'user_mentions' in ind_data['tweet'].keys() and len(ind_data['tweet']['user_mentions']) > 1:
                    #     for user in ind_data['tweet']['user_mentions']:
                    #         if user['user_id'] == -1:
                    #             print(f"cur user is fucked lol {user}")
                    #         if 'retweet_user_id' in ind_data['tweet'].keys():
                    #             print(ind_data['tweet']['retweet_user_id'])
                    # break
                    used_retweet = True
                    if 'retweet_user_id' in ind_data['tweet'].keys():
                        used_retweet = False
                        retweet_user = ind_data['tweet']['retweet_user_id']
                        ## we cannot get the twitterids of the users who tweeted this tweet through "-tweet.json"
                        ## may have to parse through account.json
                        retweet_dict[retweet_user] += 1
                    if 'user_mentions' in ind_data['tweet'].keys():
                        for user in ind_data['tweet']['user_mentions']:
                            user_id = user['user_id']
                            twitterid = user['screen_name']
                            if user_id == '-1' or '-' in user_id:
                                bad_userid += 1
                                if not used_retweet:
                                    mentions_dict[ind_data['tweet']['retweet_user_id']] += 1
                                    used_retweet = True
                                else:
                                    suspended_users['twitterid'] += 1
                                    print(f"Error in data, unknown mentioned users with -1 as userid: {user}")
                                    print(f"All mentioned: {ind_data['tweet']['user_mentions']}")
                                    print(f"This is the retweeted user: {ind_data['tweet']['retweet_user_id']}")
                            else:
                                mentions_dict[user_id] += 1

                            if user_id not in user_name_dict:
                                user_name_dict[user_id] = [twitterid]
                            elif twitterid not in user_name_dict[user_id]:
                                if user_id != '-1' and '-' not in user_id:
                                    print(f"multiple twitterids for: {user_id}: {twitterid}")
                                # print(user)
                                user_name_dict[user_id].append(twitterid)
                            

    print(f"users in mention with -1 userid: {bad_userid}")
    return


In [5]:
folder_prefix_lst = ['August_2022/TMRC14_APAC_1/', 'August_2022/TMRC14_APAC_2/', 'October_2022/TMRC15_APAC_3/']
mentions_dict_lst = []
retweet_dict_lst = []
suspended_users_lst = []
for folder in folder_prefix_lst:
    print(folder)
    mentions_dict = defaultdict(int)
    retweet_dict = defaultdict(int)
    suspended_users = defaultdict(int)
    process(folder, user_name_dict, mentions_dict, retweet_dict, suspended_users)
    mentions_dict_lst.append(mentions_dict)
    retweet_dict_lst.append(retweet_dict)
    suspended_users_lst.append(suspended_users)

August_2022/TMRC14_APAC_1/
number of files: 1000
users in mention with -1 userid: 1867
August_2022/TMRC14_APAC_2/
number of files: 1000
users in mention with -1 userid: 2710
October_2022/TMRC15_APAC_3/
number of files: 1000
multiple twitterids for: 1318636587203350530: tusdio
users in mention with -1 userid: 2289


In [11]:
for susdict in suspended_users_lst:
        print(len(susdict.keys()))

0
0
0


In [12]:
neg_lst = set(user_name_dict['-1'])
print(neg_lst)
print(len(user_name_dict))
for k in user_name_dict.keys():
    if k == '-1':
        continue
    if len(user_name_dict[k]) > 1:
        print(user_name_dict[k])
    for user in user_name_dict[k]:
        if user in neg_lst:
            print(f"userid for {user} is actually {k}")


{'dolcevenu', 'fembrutale', 'zhengjingren66', 'AktherIshrat', 'ZouYueTweets', 'LadakhBjp', 'nikel131', 'XAaqir', 'Mohd_BM_', 'Gabbar_Alp', 'HinaAhmad01', 'FangziHouse', 'moontenie', 'sherrkook', 'cungngoso', 'stigmaaa30', 'xwcnglucas', 'martienita_', 'cde_kyle', 'hombritorojo', 'jjkstyl', 'twlvesnowflakes', 'RaheelK53853161', 'urunicornnnn', 'CANOBoficial_en', 'IamPS00', 'Panyjunjun', 'Ask_Sharma_Ji', 'ClashingThought', 'SarukKhan89790', 'softiemagu', 'UK__08', 'floerlilie_', 'syo_syo_7', 'vikasirshad', 'Srjmxrell_', 'matchagreen99', 'sukaeskrimduren', 'BAD__BOY__ATIF', 'dizlalala', 'ilhamzrizky', 'starveforkoo', 'thisisnotadial', 'Underground_RT', 'ladyfawns', 'tyaszwoo', 'neshtlvl', 'kul_786', 'igoboijigobolgo', 'KalsoomNiaz', 'DominicDavidGYS', 'bbangghaje_', 'thisbasthard', 'Khan_Hilal007', 'SnehaDu98841876', 'Rakeshk26962362', 'dayah170787', 'angry_rhinos', 'ashiqrashid_', 'alcskycake', 'ed_ulrich13', 'mU5SM55kGRiBjl5', 'Geronim18601919', 'leonbakhan', 'hwhjnx', 'Bonan_Me', 'ramma

In [13]:
sorted_mentions_dict_lst = []
sorted_retweet_dict_lst = []
for d in mentions_dict_lst:
    sorted_mentions_dict_lst.append(dict(sorted(d.items(), key=lambda item: item[1], reverse=True)))
for d in retweet_dict_lst:
    sorted_retweet_dict_lst.append(dict(sorted(d.items(), key=lambda item: item[1], reverse=True)))

In [16]:
mentions_df_lst = []
retweet_df_lst = []
for d in sorted_mentions_dict_lst:
    mentions_df_lst.append(pd.DataFrame(d.items(), columns=["userid", "count"]))
for d in sorted_retweet_dict_lst:
    retweet_df_lst.append(pd.DataFrame(d.items(), columns=["userid", "count"]))

In [17]:
i = 0
name = ['TMRC14_APAC_1', 'TMRC14_APAC_2', 'TMRC15_APAC_3']
for df in retweet_df_lst:
    df.to_csv(f"~/Coding/buntain/retweet_data/{name[i]}.csv", index=False)
    i+=1
i = 0
for df in mentions_df_lst:
    df.to_csv(f"~/Coding/buntain/mentioned_data/{name[i]}.csv", index=False)
    i+=1


In [18]:
print(user_name_dict)
twitterid_df = pd.DataFrame(user_name_dict.items(), columns = ["userid", "twitterid"])
# twitterid_df
twitterid_df.to_csv("~/Coding/buntain/retweet_data/All_user_dict.csv", index=False)

{206027550: ['nessniven'], 898517260033290240: ['2VB3DcLYobExE3v'], 620912741: ['qujianming1'], 791015242448850945: ['klausv02'], 769790067183190016: ['SamanthxBerg'], 902777994829336576: ['gHk39king'], 898581276063617025: ['y6Hcuj97hRQd9gl'], 1084444722: ['nurlanyc5sr'], 761109495187513344: ['SheldonxLill'], 910296636294877184: ['xiaolaoshubai'], 902806427995172865: ['cUiIRrOVbn2Ripa'], 34649696: ['kaiking89'], 718766229700227072: ['RanvijaySowle'], 920467059175124992: ['Gregory22A'], 902779178180890626: ['HRiB831Jam'], 760761981854883840: ['honyBurchr'], 1085775014270459906: ['bronte_megan'], 3388184553: ['lacy6ysjodin'], 912171517076807680: ['oksananikolenk5'], 747309343624704000: ['kotovaninel89'], 4057984825: ['baillopud5'], 910162184772059136: ['ailaiyi1'], 918004985513705473: ['Davio91Guerl'], 788221364: ['zhangbide9600'], 926731550313480192: ['yszdian'], 2705083230: ['12muchum'], 1017843752147222530: ['masminge'], 747356042246291456: ['tattazueva2'], 2329240622: ['kristinaeug5k